In [ ]:
from openai import OpenAI
import json
from dotenv import load_dotenv
import os
from pymed import PubMed

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found. Check your .env file.")

client = OpenAI(api_key=api_key)

def pubmed_search(query, max_results=5):
    pubmed = PubMed(tool="MyTool", email="sharozhaseeb1@gmail.com")
    results = pubmed.query(query, max_results=max_results)

    result_list = []
    for article in results:
        child_dict = {}
        child_dict['title'] = article.title
        child_dict['authors'] = [((author['lastname'] or '') + ', ' + (author['firstname'] or '')).strip(', ') for author in article.authors]
        child_dict['abstract'] = article.abstract
        child_dict['publication_date'] = article.publication_date
        result_list.append(child_dict)
    
    return result_list

tools = [{
    "type": "function",
    "function": {
        "name": "pubmed_search",
        "description": "Searches PubMed for research articles based on a given query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The keyword to search for."
                },
                "max_results": {
                    "type": "integer",
                    "description": "The maximum number of results to retrieve.",
                    "default": 5
                }
            },
            "required": ["query"]
        }
    }
}]

messages = [{"role": "system", "content": "You are a research Assistant. Your task is to search PubMed for research articles based on a given query. You will then provide a summary of the articles found, including the title, authors, abstract, and publication date. Your output should be formatted as specified in the instructions. Remember to maintain a professional tone and ensure that your language is clear and concise."},
            {"role": "user", "content": "I work in machine learning specifically transformers, can you get me some research ideas?"}]
            # {"role": "user", "content": "Hi, who are you ?"}]


completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    # tool_choice="auto"
)


In [7]:
print(completion.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I am a research assistant designed to help you find and summarize research articles based on specific queries using PubMed. Let me know if there's a topic or specific area of research you'd like to explore, and I can help you find relevant information.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))


In [ ]:
completion.choices[0].finish_reason

#output for tool call will be 'tool_calls'
#output for chat will be 'stop'

'tool_calls'

In [4]:
tool_calls = completion.choices[0].message.tool_calls

for tool_call in tool_calls:
    print(tool_call.to_dict())



{'id': 'call_YSpCPU6rXrguFVNQAFMhAF9x', 'function': {'arguments': '{"query":"transformers machine learning","max_results":5}', 'name': 'pubmed_search'}, 'type': 'function'}


In [ ]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = pubmed_search(args["query"], args["max_results"])

In [ ]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(json.dumps(result, indent = 4))
})

completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)